In [ ]:
from abc import ABC
import types
import uuid
from pydantic import BaseModel, Field, computed_field
from pydantic._internal._model_construction import ModelMetaclass
import typing
import inspect
from functools import wraps
import enum
from types import FunctionType


class BoundMethodInfo(BaseModel):
    model_config = {"arbitrary_types_allowed": True}
    
    method_type: typing.Type
    method: typing.Any
    wrapped: typing.Optional[typing.Callable] = Field(default=None)
    signature: inspect.Signature
    bounded_class: typing.Optional[typing.Type]

    @staticmethod
    def inspect(method: typing.Any) -> typing.Optional["BoundMethodInfo"]:
        """
        Inspect a method to get its type, signature, and bounded class.

        :param method: The method to inspect.
        :type method: Any
        :return: A MethodInfo object containing the method type, signature, and bounded class.
        :rtype: MethodInfo
        :raises ValueError: If the method is not a classmethod, staticmethod, or function.
        """
        def _get_bounded_class(m: FunctionType|classmethod):
            def _recure_get_bounded_class(parent_obj, qual_name):
                child_name = qual_name.pop(0)
                parent_obj = getattr(parent_obj, child_name)
                if not len(qual_name):
                    return parent_obj 
                return _recure_get_bounded_class(parent_obj, qual_name)
            
            module = inspect.getmodule(m)
            qual_name = m.__qualname__.split(".")[:-1]
            if not qual_name:
                return None
            return _recure_get_bounded_class(module, qual_name)

        def _is_static_method(m: FunctionType, cls: typing.Type):
            for cls in inspect.getmro(cls):
                if inspect.isroutine(m):
                    print(f"{m.__name__ in cls.__dict__=}, {m=}, {cls.__dict__=}")
                    if m.__name__ in cls.__dict__:
                        if isinstance(m, staticmethod):
                            return True
            return False
        
        bounded_class = _get_bounded_class(method)
        if bounded_class is None:
            raise ValueError(f"Method {method} is not bounded to any class.")
        
        if hasattr(method, "__wrapped__"):
            wrapped = method.__wrapped__
            method_type = staticmethod if _is_static_method(method, bounded_class) else classmethod
        else:
            wrapped = method
            method_type = FunctionType

        signature = inspect.signature(wrapped)

        return BoundMethodInfo(
            method_type=method_type, 
            method=method, 
            wrapped=wrapped, 
            signature=signature,
            bounded_class=bounded_class
            )



class MethodSignatureDispatcher(BaseModel):
    containing_cls_: typing.Optional[typing.Type] = Field(None, description="class container decorated method(s)")
    callable_type_: typing.Optional[typing.Type] = Field(None, description="class container decorated method(s)")
    decorated : typing.Optional[typing.Callable] = Field(None, description="decorated method")
    overloads : list[typing.Optional[typing.Callable]] = Field([], description="decorated method")

    def __init__(self, *args, **kwargs):
        if len(args) == 1:
            method_info = BoundMethodInfo.inspect(args[0])
            if not method_info:
                raise ValueError(f"{type(self).__name__} can only be used to decorate \
{FunctionType.__name__}, {classmethod.__name__} and {staticmethod.__name__}.")
            super().__init__()
            self.decorated = method_info.method
            if "add_to_dispatch" in kwargs:
                self.overloads.append(method_info.method if method_info.method_type is not classmethod else method_info.wrapped)
    
    def __call__(self, *args, **kwargs) -> typing.Self:
        if not hasattr(self, "decorated"):
            self.__init__(*args, **kwargs)
            return self
        return self.decorated( *args, **kwargs)


    def overload(self, func):
        if callable(func):
            pass

# signature_dispatch = MethodSignatureDispatcher

class Observer(BaseModel):
    model_config = {"arbitrary_types_allowed": True}

    func: typing.Optional[typing.Callable] = Field(None, description="function to be observed")
    call_type: typing.Optional[str] = Field(None, description="call type of the function")
    args: tuple = Field(description="args to be observed")
    kwargs: dict = Field(description="kwargs to be observed")

    @computed_field
    def module(self) -> typing.Optional[types.ModuleType]:
        if self.func is None:
            return None
        return inspect.getmodule(self.func)

    @computed_field
    def qualname(self) -> typing.Optional[str]:
        if self.func is None:
            return None
        return self.func.__qualname__
    
    def start(self):
        """
        Start observing the function.
        """
        

def signature_dispatcher(*args, **kwargs):
    """
    A decorator to observe the signature of a function or method.
    It can be used with or without arguments.
    If used with arguments, it will return a decorator that can be applied to a function.
    If used without arguments, it will return a function that can be called directly.
    """
    if len(args) == 1 and len(kwargs) == 0:
        func = args[0]
        Observer(
            func=func,
            call_type="@signature_dispatcher",
            args=args,
            kwargs=kwargs
        ).start()

        return func
    
    if len(args) == 0 and len(kwargs) == 0:
        def decorator(func):
            Observer(
                func=func,
                call_type="@signature_dispatcher()",
                args=args,
                kwargs=kwargs
            ).start()

            return func
        return decorator
    
    if len(args) == 0 and len(kwargs) > 0:
        def decorator(func):
            Observer(
                func=func,
                call_type="@signature_dispatcher(...)",
                args=args,
                kwargs=kwargs
            ).start()

            return func
        return decorator
    

SyntaxError: unterminated string literal (detected at line 172) (1761184987.py, line 172)

In [ ]:
class test_class_3:
    @signature_dispatcher()
    @classmethod
    def testing_classmethod(cls, *args, **kwargs):
        """ABC"""
        print(f"test.testing_classmethod(self, *{args}, **{kwargs})")

    @signature_dispatcher()
    @staticmethod
    def testing_static(*args, **kwargs):
        print(f"test.testing_static(self, *{args}, **{kwargs})")



In [25]:
test_class_3.testing_classmethod(1, 2, 3)

test.testing_classmethod(self, *(1, 2, 3), **{})
